# Interactive SAM2 Point and Box Selection

This notebook provides an interactive interface for selecting points and bounding boxes on images for SAM2 segmentation.

## Features
- Click to select points
- Click and drag to select bounding boxes
- Visual feedback with coordinate display
- Export coordinates for use with SAM2

## Usage
1. Run all cells
2. Load your image
3. Select interaction mode (point or box)
4. Click/drag on the image to make selections
5. Copy the generated command to run SAM2 segmentation

## Import Required Libraries

In [1]:
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from matplotlib.widgets import Button, RadioButtons
import numpy as np
from PIL import Image
import os
import ipywidgets as widgets
from IPython.display import display, HTML
import json

## Setup Interactive Backend

In [3]:
# Enable interactive matplotlib in Jupyter
%matplotlib inline

# Configure matplotlib for better display
plt.rcParams['figure.figsize'] = [12, 8]
plt.rcParams['font.size'] = 10

print("✅ Matplotlib configured with inline backend")

✅ Matplotlib configured with inline backend


## Configuration and Image Loading

In [4]:
# Configuration
# Update these paths according to your setup
RENDERED_RESULTS_PATH = "/usr/project/Feature4X/output/bear/final_viz/81_round_moving/rendered_results.pth"
VIDEO_DIR = "/usr/project/Feature4X/output/bear/final_viz/81_round_moving_rgb"
SAVE_DIR = "../try_sam_interactive_notebook"
FRAME_INDEX = 0  # Which frame to use for selection

# Create save directory if it doesn't exist
os.makedirs(SAVE_DIR, exist_ok=True)

print(f"Rendered results path: {RENDERED_RESULTS_PATH}")
print(f"Video directory: {VIDEO_DIR}")
print(f"Save directory: {SAVE_DIR}")
print(f"Frame index: {FRAME_INDEX}")

Rendered results path: /usr/project/Feature4X/output/bear/final_viz/81_round_moving/rendered_results.pth
Video directory: /usr/project/Feature4X/output/bear/final_viz/81_round_moving_rgb
Save directory: ../try_sam_interactive_notebook
Frame index: 0


In [5]:
# Load the image for selection
def load_frame_image(video_dir, frame_index=0):
    """Load the frame image from video directory."""
    frame_files = [f for f in os.listdir(video_dir) 
                   if f.lower().endswith(('.jpg', '.jpeg', '.png'))]
    
    if not frame_files:
        raise ValueError(f"No image files found in {video_dir}")
    
    # Sort files to get consistent ordering
    frame_files.sort()
    
    if frame_index >= len(frame_files):
        frame_index = 0
        print(f"Frame index out of range, using frame 0")
    
    frame_path = os.path.join(video_dir, frame_files[frame_index])
    image = Image.open(frame_path)
    
    print(f"Loaded frame: {frame_files[frame_index]}")
    print(f"Image size: {image.size[0]}x{image.size[1]} (width x height)")
    
    return image, frame_files[frame_index]

# Load the image
try:
    image, frame_name = load_frame_image(VIDEO_DIR, FRAME_INDEX)
    print("✅ Image loaded successfully!")
except Exception as e:
    print(f"❌ Error loading image: {e}")
    print("Please check the VIDEO_DIR path and make sure it contains image files.")
    image = None

Loaded frame: 0000.jpg
Image size: 854x480 (width x height)
✅ Image loaded successfully!


## Interactive Selection Class

In [6]:
class SimpleSAM2Selector:
    def __init__(self, image):
        self.image = image
        self.points = []
        self.boxes = []
        self.current_selections = {'points': [], 'boxes': []}
        
    def show_image_with_selections(self, title="Image with Selections"):
        """Display the image with current selections."""
        fig, ax = plt.subplots(figsize=(14, 10))
        ax.imshow(self.image)
        ax.set_title(title)
        ax.grid(True, alpha=0.3)
        ax.set_xlabel('X coordinate')
        ax.set_ylabel('Y coordinate')
        
        # Draw points
        for i, point in enumerate(self.points):
            circle = plt.Circle((point[0], point[1]), radius=8, color='red', zorder=10)
            ax.add_patch(circle)
            ax.text(point[0] + 15, point[1] - 15, f'P{i+1}({point[0]},{point[1]})', 
                   color='red', fontweight='bold', fontsize=10, zorder=11)
        
        # Draw boxes
        for i, box in enumerate(self.boxes):
            width = box[2] - box[0]
            height = box[3] - box[1]
            rect = plt.Rectangle((box[0], box[1]), width, height,
                               linewidth=3, edgecolor='green', 
                               facecolor='none', zorder=10)
            ax.add_patch(rect)
            ax.text(box[0], box[1] - 20, 
                   f'B{i+1}({box[0]},{box[1]},{box[2]},{box[3]})',
                   color='green', fontweight='bold', fontsize=10, zorder=11)
        
        plt.tight_layout()
        plt.show()
        
        return fig, ax
    
    def add_point(self, x, y):
        """Add a point selection."""
        self.points.append([int(x), int(y)])
        print(f"✅ Point {len(self.points)} added: ({int(x)}, {int(y)})")
        
    def add_box(self, x1, y1, x2, y2):
        """Add a box selection."""
        box = [min(int(x1), int(x2)), min(int(y1), int(y2)), 
               max(int(x1), int(x2)), max(int(y1), int(y2))]
        self.boxes.append(box)
        print(f"✅ Box {len(self.boxes)} added: {box}")
        
    def clear_points(self):
        """Clear all points."""
        self.points = []
        print("🗑️ All points cleared")
        
    def clear_boxes(self):
        """Clear all boxes."""
        self.boxes = []
        print("🗑️ All boxes cleared")
        
    def clear_all(self):
        """Clear all selections."""
        self.points = []
        self.boxes = []
        print("🗑️ All selections cleared")
        
    def get_selections(self):
        """Get current selections."""
        return {
            'points': self.points.copy(),
            'boxes': self.boxes.copy()
        }

# Helper functions for easy interaction
def show_interactive_help():
    print("🎯 INTERACTIVE SAM2 SELECTOR HELP")
    print("=" * 50)
    print("After creating the selector, use these methods:")
    print()
    print("📍 Adding Selections:")
    print("  selector.add_point(x, y)           # Add a point at coordinates (x, y)")
    print("  selector.add_box(x1, y1, x2, y2)   # Add a box from (x1,y1) to (x2,y2)")
    print()
    print("🖼️ Viewing:")
    print("  selector.show_image_with_selections()  # Show image with current selections")
    print()
    print("🗑️ Clearing:")
    print("  selector.clear_points()             # Remove all points")
    print("  selector.clear_boxes()              # Remove all boxes") 
    print("  selector.clear_all()                # Remove everything")
    print()
    print("📊 Getting Results:")
    print("  selector.get_selections()           # Get current points and boxes")
    print()
    print("💡 Example workflow:")
    print("  1. Create selector: selector = SimpleSAM2Selector(image)")
    print("  2. View image: selector.show_image_with_selections()")
    print("  3. Add selections: selector.add_point(200, 150)")
    print("  4. Add box: selector.add_box(100, 100, 300, 300)")
    print("  5. View updated: selector.show_image_with_selections()")
    print("  6. Get results: Run the 'Display Selected Data' cell")

print("📝 SimpleSAM2Selector class defined!")
print("Use show_interactive_help() to see usage instructions.")

📝 SimpleSAM2Selector class defined!
Use show_interactive_help() to see usage instructions.


## Create Interactive Plot

In [ ]:
if image is not None:
    # Create simplified selector
    selector = SimpleSAM2Selector(image)
    
    print("🎯 Interactive selector created!")
    print("? Image Information:")
    print(f"   Size: {image.size[0]}x{image.size[1]} (width x height)")
    print(f"   Frame: {frame_name}")
    
    # Show the initial image
    selector.show_image_with_selections("Initial Image - Ready for Selection")
    
    print("\n📋 Quick Start Instructions:")
    print("1. Note the coordinates from the image above")
    print("2. Use the methods below to add selections:")
    print("   • selector.add_point(x, y)")  
    print("   • selector.add_box(x1, y1, x2, y2)")
    print("3. View updates with: selector.show_image_with_selections()")
    print("4. Run the next cell to generate SAM2 commands")
    print()
    print("💡 For detailed help, run: show_interactive_help()")
    
    # Add some example buttons using ipywidgets
    from ipywidgets import interact, interactive, fixed, interact_manual
    import ipywidgets as widgets
    
    # Create interactive widgets for easy coordinate input
    print("\\n🎮 INTERACTIVE CONTROLS:")
    
    # Point input widget
    def add_point_widget(x=200, y=150):
        selector.add_point(x, y)
        selector.show_image_with_selections()
    
    # Box input widget  
    def add_box_widget(x1=100, y1=100, x2=300, y2=300):
        selector.add_box(x1, y1, x2, y2)
        selector.show_image_with_selections()
    
    print("Use the interactive widgets below to add selections:")

else:
    print("❌ Cannot create interactive selector without a valid image.")
    print("Please check the image loading cell above.")

In [ ]:
# Interactive Widgets for Easy Selection
if image is not None and 'selector' in locals():
    from ipywidgets import interact, widgets
    from IPython.display import display
    
    print("🎮 INTERACTIVE SELECTION WIDGETS")
    print("=" * 40)
    
    # Point selection widget
    @interact(
        x=widgets.IntSlider(min=0, max=image.size[0]-1, step=1, value=200, description='Point X:'),
        y=widgets.IntSlider(min=0, max=image.size[1]-1, step=1, value=150, description='Point Y:')
    )
    def add_point_interactive(x, y):
        pass  # The interaction happens through the widget
    
    # Buttons for point operations
    add_point_btn = widgets.Button(description="Add Point", button_style='success')
    clear_points_btn = widgets.Button(description="Clear Points", button_style='warning')
    
    def on_add_point(b):
        x = add_point_interactive.widget.children[0].value
        y = add_point_interactive.widget.children[1].value
        selector.add_point(x, y)
        selector.show_image_with_selections()
    
    def on_clear_points(b):
        selector.clear_points()
        selector.show_image_with_selections()
    
    add_point_btn.on_click(on_add_point)
    clear_points_btn.on_click(on_clear_points)
    
    print("\\n📍 Point Selection:")
    display(widgets.HBox([add_point_btn, clear_points_btn]))
    
    # Box selection widget
    print("\\n📦 Box Selection - Enter coordinates:")
    box_x1 = widgets.IntText(value=100, description='X1 (left):')
    box_y1 = widgets.IntText(value=100, description='Y1 (top):')
    box_x2 = widgets.IntText(value=300, description='X2 (right):')
    box_y2 = widgets.IntText(value=300, description='Y2 (bottom):')
    
    add_box_btn = widgets.Button(description="Add Box", button_style='success')
    clear_boxes_btn = widgets.Button(description="Clear Boxes", button_style='warning')
    
    def on_add_box(b):
        selector.add_box(box_x1.value, box_y1.value, box_x2.value, box_y2.value)
        selector.show_image_with_selections()
    
    def on_clear_boxes(b):
        selector.clear_boxes()
        selector.show_image_with_selections()
    
    add_box_btn.on_click(on_add_box)
    clear_boxes_btn.on_click(on_clear_boxes)
    
    display(widgets.HBox([box_x1, box_y1]))
    display(widgets.HBox([box_x2, box_y2]))
    display(widgets.HBox([add_box_btn, clear_boxes_btn]))
    
    # Quick actions
    show_btn = widgets.Button(description="Show Current Selections", button_style='info')
    clear_all_btn = widgets.Button(description="Clear All", button_style='danger')
    
    def on_show(b):
        selector.show_image_with_selections()
    
    def on_clear_all(b):
        selector.clear_all()
        selector.show_image_with_selections()
    
    show_btn.on_click(on_show)
    clear_all_btn.on_click(on_clear_all)
    
    print("\\n🎯 Quick Actions:")
    display(widgets.HBox([show_btn, clear_all_btn]))
    
else:
    print("⚠️ Selector not available. Please run the previous cells first.")

## Display Selected Data and Generate Commands

In [ ]:
def generate_sam2_commands(points, boxes, rendered_results_path, save_dir, frame_index=0):
    """Generate SAM2 command line commands based on selections."""
    commands = []
    base_cmd = f"python sam2_segmentation.py --rendered_results_path {rendered_results_path} --frame {frame_index} --save_dir {save_dir}"
    
    # Commands for points only
    for i, point in enumerate(points):
        cmd = f"{base_cmd} --point {point[0]} {point[1]}"
        commands.append((f"Point {i+1}", cmd))
    
    # Commands for boxes only
    for i, box in enumerate(boxes):
        cmd = f"{base_cmd} --box {box[0]} {box[1]} {box[2]} {box[3]}"
        commands.append((f"Box {i+1}", cmd))
    
    # Combined commands (if both points and boxes exist)
    if points and boxes:
        for i, point in enumerate(points):
            for j, box in enumerate(boxes):
                cmd = f"{base_cmd} --point {point[0]} {point[1]} --box {box[0]} {box[1]} {box[2]} {box[3]}"
                commands.append((f"Point {i+1} + Box {j+1}", cmd))
    
    return commands

if image is not None and 'selector' in locals():
    # Get current selections
    selections = selector.get_selections()
    points = selections['points']
    boxes = selections['boxes']
    
    print("📊 CURRENT SELECTIONS:")
    print("=" * 50)
    
    if points:
        print(f"\n🔴 Points ({len(points)}):")
        for i, point in enumerate(points):
            print(f"  Point {i+1}: ({point[0]}, {point[1]})")
    else:
        print("\n🔴 Points: None selected")
    
    if boxes:
        print(f"\n🟢 Boxes ({len(boxes)}):")
        for i, box in enumerate(boxes):
            print(f"  Box {i+1}: [{box[0]}, {box[1]}, {box[2]}, {box[3]}] (x1, y1, x2, y2)")
    else:
        print("\n🟢 Boxes: None selected")
    
    # Generate and display commands
    if points or boxes:
        print("\n🚀 SAM2 COMMANDS:")
        print("=" * 50)
        
        commands = generate_sam2_commands(points, boxes, RENDERED_RESULTS_PATH, SAVE_DIR, FRAME_INDEX)
        
        for desc, cmd in commands:
            print(f"\n# {desc}")
            print(cmd)
        
        # Save selections to JSON
        selection_file = os.path.join(SAVE_DIR, "interactive_selections.json")
        with open(selection_file, 'w') as f:
            json.dump({
                'frame_name': frame_name,
                'frame_index': FRAME_INDEX,
                'image_size': [image.size[0], image.size[1]],
                'points': points,
                'boxes': boxes,
                'commands': [cmd for _, cmd in commands]
            }, f, indent=2)
        
        print(f"\n💾 Selections saved to: {selection_file}")
        
        # Create a simple execution script
        if commands:
            script_file = os.path.join(SAVE_DIR, "run_sam2.sh")
            with open(script_file, 'w') as f:
                f.write("#!/bin/bash\n\n")
                f.write("# Auto-generated SAM2 execution script\n")
                f.write("# Generated from interactive notebook selections\n\n")
                f.write("cd /usr/project/Feature4X/sam2\n\n")
                
                for desc, cmd in commands:
                    f.write(f"# {desc}\n")
                    f.write(f"conda run -n feature4x {cmd}\n\n")
            
            # Make script executable
            import stat
            os.chmod(script_file, stat.S_IRWXU | stat.S_IRGRP | stat.S_IROTH)
            
            print(f"🔧 Execution script saved to: {script_file}")
            print("\n▶️  To run all commands:")
            print(f"   chmod +x {script_file} && {script_file}")
        
    else:
        print("\n⚠️  No selections made yet.")
        print("Use the interactive plot above to select points or boxes.")

else:
    print("❌ Interactive selector not available. Please run the previous cells first.")

## Quick Command Execution (Optional)

In [ ]:
# Uncomment and run this cell to execute the first SAM2 command directly
# WARNING: This will run SAM2 segmentation which may take some time

# import subprocess
# import sys

# if image is not None and 'selector' in locals():
#     selections = selector.get_selections()
#     points = selections['points']
#     boxes = selections['boxes']
#     
#     if points or boxes:
#         commands = generate_sam2_commands(points, boxes, RENDERED_RESULTS_PATH, SAVE_DIR, FRAME_INDEX)
#         
#         if commands:
#             print(f"🔄 Executing: {commands[0][1]}")
#             
#             # Change to sam2 directory and run command
#             try:
#                 result = subprocess.run(
#                     ["conda", "run", "-n", "feature4x"] + commands[0][1].split()[1:],
#                     cwd="/usr/project/Feature4X/sam2",
#                     capture_output=True,
#                     text=True,
#                     timeout=300  # 5 minute timeout
#                 )
#                 
#                 print("✅ Command completed!")
#                 print("STDOUT:", result.stdout[-1000:])  # Last 1000 chars
#                 if result.stderr:
#                     print("STDERR:", result.stderr[-1000:])  # Last 1000 chars
#                     
#             except subprocess.TimeoutExpired:
#                 print("⏰ Command timed out after 5 minutes")
#             except Exception as e:
#                 print(f"❌ Error executing command: {e}")
#     else:
#         print("⚠️ No selections to execute")
# else:
#     print("❌ Cannot execute without valid selections")

print("💡 Uncomment the code above to execute SAM2 commands directly from the notebook")

## Summary and Next Steps

This interactive notebook provides a user-friendly interface for selecting points and bounding boxes on images for SAM2 segmentation.

### What this notebook does:
1. **Interactive Selection**: Click to select points, drag to create boxes
2. **Visual Feedback**: See your selections highlighted on the image
3. **Command Generation**: Automatically generates SAM2 command line instructions
4. **Export Options**: Saves selections to JSON and creates executable scripts

### To use your selections:
1. Copy any of the generated commands from the output above
2. Run them in a terminal in the `/usr/project/Feature4X/sam2` directory
3. Or use the generated shell script for batch execution

### Tips:
- **Point Selection**: Best for clicking on specific objects or features
- **Box Selection**: Best for defining rectangular regions around objects
- **Combined**: You can use both points and boxes for more precise control
- **Multiple Selections**: Create multiple points/boxes for different segmentation tasks

### File Outputs:
- `interactive_selections.json`: Your selections in JSON format
- `run_sam2.sh`: Executable script with all generated commands
- SAM2 will create additional outputs when you run the segmentation commands